In [77]:
from constants import *
import subprocess
import json
import os
from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI
from eth_account import Account

In [78]:
# Load dot env
load_dotenv()

True

In [79]:
#Generating a Mnemonic
mnemonic = os.getenv('MNEMONIC')

In [80]:
# Create coin object
class coin:
    BTC = 'btc'
    ETH = 'eth'
    BTCTEST = 'btc-test'

In [81]:
#Deriving the wallet keys
def derive_wallets (mnemonic,coin,number):
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php --mnemonic="{mnemonic}" -g --numderive="{number}" --coin="{coin}" --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) =p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [82]:
# Printing the wallet keys
derive_wallets(mnemonic,ETH,3)

[{'address': '0x3b3cb82b8b706757c78bd02f00B052E8e1929e04',
  'index': 0,
  'path': "m/44'/60'/0'/0/0",
  'privkey': '0xcbf6bba94177a615b6cd0fd434a1cf3fb62b84abf1a28e7d3b7b4788b567dee0',
  'pubkey': '03dc98bd00fb18a08817df0c68de42c3cc0dc4e1c93226ed6d38b73a98c8bb8b97',
  'pubkeyhash': '3c07122eea28222a09d5915739126013d1b11991',
  'xprv': 'xprvA3cnv9vAEFaet5HBf3mw3z92GQJJZBorkuPjTQSzWvsWGDvKksqKCPZmPNczKcGFWuG9KJMc6RrmWK9SPuUi7ewve2kjt71bXmgxXytFcTz',
  'xpub': 'xpub6Gc9KfT44d8x6ZMem5JwR85kpS8nxeXi88KLFnrc5GQV92FUJR9ZkBtFEfipoZDg4bPbRv4YQfQUtZoUTroDxZiY8NJma3T3mEmePD2P3sU'},
 {'address': '0x8B2071CFC67f596BA4C280356bFeED43C030A5c6',
  'index': 1,
  'path': "m/44'/60'/0'/0/1",
  'privkey': '0x6153f62d4c7ab191aee7bccfab58e853f892107d559b94d20837689630f0873e',
  'pubkey': '024ea707c59f680aa08653829ff36c3f474860695fe5ae4240d08d1312331c61ca',
  'pubkeyhash': 'c67f4d25e52a6b67f2986fdfce4feb49d428ad24',
  'xprv': 'xprvA3cnv9vAEFaev899MLK6AjHtjPTYVVpdh3vboiiPe8mSZBRYgnAezfNGtPyUgk22EFSs2RoD1EdSoJ

In [83]:
# Linking the transaction signing libraries
def priv_key_to_account (coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [84]:

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [85]:
eth_acc = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH,5)[0]['privkey'])
btc_acc = priv_key_to_account(BTCTEST, derive_wallets(mnemonic,BTCTEST,5)[0]['privkey'])

In [86]:
eth_acc

In [87]:
#btc_acc.address

In [88]:
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
        {"from": eth_acc.address, "to": to, "value": amount}
    )
        return {
            "from": eth_acc.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
            
    }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [89]:
#key = wif_to_key("cPSgcKgTyA3jf2RBANPoUojz4CcNfcHGxto2t9LyPcArXS5bkFUp")
#key.get_balance(ETH)

In [90]:
#key.get_balance("usd")

In [91]:
create_tx('ETH',eth_acc,'0x8B2071CFC67f596BA4C280356bFeED43C030A5c6',1250000000000000000000)

In [92]:
#Sending currency (note amount is in WEI)
def send_tx(coin,account, recipient, amount):
    tx = create_tx(coin,account,recipient,amount)
    
    if coin == ETH:
        signed_tx = eth_account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        signed_tx = btc_account.sign_transaction(tx)
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [93]:
create_tx('ETH',eth_acc,'0x8B2071CFC67f596BA4C280356bFeED43C030A5c6',1250000000000000000000)

In [94]:
send_tx(ETH,eth_acc,'0x8B2071CFC67f596BA4C280356bFeED43C030A5c6',1250000000000000000000)

'0x8e01a223b6ee71b620935ab76e1a970ac7aa9f039d66df9507798752e49454be'

In [95]:
send_tx(BTCTEST,btc_acc,'n3sLNayxAmKgcdERiGDkedbhS55cow3t6P',.001)

In [96]:
#Creating easier access to addresses from the index
def transaction(coin, sender, to, amount):
    send_key = derive_wallets(mnemonic, coin, 5)[sender]['privkey']
    send_hash = priv_key_to_account(coin, send_key)
    to_address = derive_wallets(mnemonic, coin, 5)[to]['address']
    return send_tx(coin, send_hash, to_address, amount)

In [97]:
#Sending a transaciton
transaction(ETH,0,1,1250000000000000000000)

'0x088dbaa02c8507849f946648476c3b7d7db193f012d8dd2915ca0345a71d27c8'